# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from xlwings.constants import AutoFillType, HAlign

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
reviews = pd.read_csv('reviews_sample.csv', header=0, sep=',')
reviews = reviews.rename(columns={'Unnamed: 0': 'id'})
reviews.head()

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [3]:
recipes = pd.read_csv('recipes_sample_with_tags_ingredients.csv', sep=',')
recipes = recipes.drop(['Unnamed: 0', 'n_steps', 'n_tags', 'tags', 'n_ingredients', 'contributor_id'], 1)
recipes.head()

,name,id,minutes,submitted,description,ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,3
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,9
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,12


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [4]:
wb = xw.Book()
sheet = wb.sheets['Лист1']
sheet.name = 'Рецепты'
wb.sheets.add('Отзывы')

<Sheet [Книга1]Отзывы>

In [5]:
sheet.range('A1').value = recipes.sample(frac=0.05)
sheet = wb.sheets['Отзывы']
sheet.range('A1').value = reviews.sample(frac=0.05)

In [6]:
wb.save('recipes.xlsx')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [7]:
sheet = wb.sheets['Рецепты']
time = sheet.range('D2:D1501').options(np.array).value
time

array([ 40.,  45., 150., ...,  40.,  30.,  25.])

In [8]:
sheet.range('H1').value = 'seconds_assign'
sheet.range('H2:H1501').options(transpose=True).value = np.nan_to_num(time * 60)

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [9]:
sheet.range('I1').value = 'seconds_formula'
sheet.range('I2').formula = '=D2*60'
sheet.range('I2').api.AutoFill(sheet.range('I2:I1501').api, AutoFillType.xlFillDefault)

True

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [10]:
sheet.range('J1').value = 'n_reviews'
sheet.range('J2').formula = '=COUNTIF(Отзывы!D$2:D$6336,C2)'
sheet.range('J2').api.AutoFill(sheet.range('J2:J1501').api, AutoFillType.xlFillDefault)

True

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [11]:
sheet.range('H1:J1501').api.Font.Bold = True
sheet.range('H1:J1501').api.HorizontalAlignment = HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [12]:
for x in sheet.range('D2:D1501'):
    if x.value < 5:
        x.color = (0, 255, 0)
    elif 5 <= x.value <= 10:
        x.color = (255, 255, 0)
    else:
        x.color = (255, 0, 0)

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [13]:
sheet = wb.sheets['Отзывы']

In [22]:
rg = wb.sheets['Рецепты'].range("C2:C1502")
def validate():
    for x in sheet.range('F2:F6337'):
        if not (0.0 <= float(x.value) and float(x.value) <= 5.0 and sheet.range(f"D{x.row}").value in rg):
            sheet.range(f"A{x.row}:G{x.row}").color = (255, 0, 0)



In [23]:
validate()

KeyboardInterrupt: 